In [1]:
import datetime
import pytz
import xbos_services_getter

In [2]:
# HOST_ADDRESSES FOR ALL MICROSERVICES
# SOON WILL BE REPLACED WITH ONE ADDRESS
# NEED TO ONLY DEFINE THE ADDRESS FOR THE SERVICES YOU USE
DISCOMFORT_HOST_ADDRESS=None #"FILL_ME_IN"
HVAC_CONSUMPTION_HOST_ADDRESS=None #"FILL_ME_IN"
TEMPERATURE_BANDS_HOST_ADDRESS=None #"FILL_ME_IN"
PRICE_HOST_ADDRESS=None #"FILL_ME_IN"
OCCUPANCY_HOST_ADDRESS=None #"FILL_ME_IN"
OUTDOOR_TEMPERATURE_HISTORICAL_HOST_ADDRESS=None #"FILL_ME_IN"
BUILDING_ZONE_NAMES_HOST_ADDRESS=None #"FILL_ME_IN"
INDOOR_DATA_HISTORICAL_HOST_ADDRESS=None #"FILL_ME_IN"
INDOOR_TEMPERATURE_PREDICTION_HOST_ADDRESS=None #"FILL_ME_IN"
METER_DATA_HISTORICAL_HOST_ADDRESS=None #"FILL_ME_IN"
OPTIMIZER_HOST_ADDRESS=None #"FILL_ME_IN"

In [3]:
# CREATE A STUB TO EACH MICROSERVICE THAT YOU USE
# THIS STUB ONLY GETS CALLED ONCE TO ESTABLISH A CONNECTION TO THE SERVICE
discomfort_stub = xbos_services_getter.get_discomfort_stub(DISCOMFORT_HOST_ADDRESS)
hvac_consumption_stub = xbos_services_getter.get_hvac_consumption_stub(HVAC_CONSUMPTION_HOST_ADDRESS)
temperature_bands_stub = xbos_services_getter.get_temperature_band_stub(TEMPERATURE_BANDS_HOST_ADDRESS)
price_stub = xbos_services_getter.get_price_stub(PRICE_HOST_ADDRESS)
occupancy_stub = xbos_services_getter.get_occupancy_stub(OCCUPANCY_HOST_ADDRESS)
outdoor_temperature_historical_stub = xbos_services_getter.get_outdoor_historic_stub(OUTDOOR_TEMPERATURE_HISTORICAL_HOST_ADDRESS)
building_zone_names_stub = xbos_services_getter.get_building_zone_names_stub(BUILDING_ZONE_NAMES_HOST_ADDRESS)
indoor_data_historical_stub = xbos_services_getter.get_indoor_historic_stub(INDOOR_DATA_HISTORICAL_HOST_ADDRESS)
indoor_temperature_prediction_stub = xbos_services_getter.get_indoor_temperature_prediction_stub(INDOOR_TEMPERATURE_PREDICTION_HOST_ADDRESS)
meter_data_historical_stub = xbos_services_getter.get_meter_data_historical_stub(METER_DATA_HISTORICAL_HOST_ADDRESS)
# optimizer_stub = xbos_services_getter.get_optimizer_stub(OPTIMIZER_HOST_ADDRESS)

In [4]:
# GET A LIST OF ALL BUILDINGS 
buildings = xbos_services_getter.get_buildings(building_zone_names_stub)
print("List of buildings:",buildings)

# GET ALL ZONES FOR EACH BUILDING
for bldg in buildings:
    zones = xbos_services_getter.get_zones(building_zone_names_stub,bldg)
    print("List of zones in building:",bldg,"are:",zones)
    
# ALTERNATIVELY YOU CAN GET A DICTIONARY OF ALL BUILDINGS AND ZONES WITH ONE CALL
buildings_zones = xbos_services_getter.get_all_buildings_zones(building_zone_names_stub)
print("All buildings and zones:",buildings_zones)

List of buildings: ['jesse-turner-center', 'ciee', 'csu-dominguez-hills', 'hayward-station-8', 'avenal-movie-theatre', 'word-of-faith-cc', 'hayward-station-1', 'avenal-veterans-hall', 'avenal-animal-shelter', 'south-berkeley-senior-center', 'local-butcher-shop', 'avenal-recreation-center', 'orinda-community-center', 'avenal-public-works-yard', 'north-berkeley-senior-center', 'berkeley-corporate-yard']
List of zones in building: jesse-turner-center are: ['hvac_zone_resource_center', 'hvac_zone_class_132', 'hvac_zone_meeting_room_122', 'hvac_zone_multi_purpose_147', 'hvac_zone_fitness_room_138', 'hvac_zone_assembly_112', 'hvac_zone_basketball_court_2', 'hvac_zone_basketball_court_6', 'hvac_zone_basketball_court_5', 'hvac_zone_lobby', 'hvac_zone_basketball_court_3', 'hvac_zone_control_desk', 'hvac_zone_foyer_gallery', 'hvac_zone_assembly_113', 'hvac_zone_class_131', 'hvac_zone_front_entrance', 'hvac_zone_office_room_148', 'hvac_zone_basketball_court_4', 'hvac_zone_dance_room_109', 'hvac_z

In [5]:
# EXAMPLE TO RETRIEVE DATA FOR ONE DAY AT ONE HOUR INTERVAL STARTING JAN 1ST 2019
start = pytz.timezone('US/Pacific').localize(datetime.datetime(year=2019, month=1, day=1, hour=0, minute=0))
end = start + datetime.timedelta(days=1)
window = "1h"

In [6]:
# TODO: DOES NOT WORK "Exception calling application: no such table: view_meter"
# GET HISTORICAL BUILDING METER & GREENBUTTON DATA FOR EACH BUILDING
# MEAN, MAX, MIN, COUNT, SUM and RAW
# start = '2019-01-01T00:00:00Z'
# end = '2019-01-02T00:00:00Z'
electric_point_type = 'Building_Electric_Meter'
greenbutton_point_type = 'Green_Button_Meter'
aggregate = 'MEAN' # MEAN, MAX, MIN, COUNT, SUM and RAW
for bldg in buildings:
    if bldg == "jesse-turner-center" or bldg == "csu-dominguez-hills" or bldg == "word-of-faith-cc":
        continue
    print(bldg)
    print(start,end)
    electric_meter_data = xbos_services_getter.get_meter_data_historical(meter_data_historical_stub,bldg,start,end,electric_point_type,aggregate,window)
    greenbutton_meter_data = xbos_services_getter.get_meter_data_historical(meter_data_historical_stub,bldg,start,end,greenbutton_point_type,aggregate,window)
    print(electric_meter_data)
    print(greenbutton_meter_data)

ciee
2019-01-01 00:00:00-08:00 2019-01-02 00:00:00-08:00
                                 power
datetime                              
2019-01-01 08:00:00+00:00  1961.536098
2019-01-01 09:00:00+00:00  1863.267176
2019-01-01 10:00:00+00:00  1822.831325
2019-01-01 11:00:00+00:00  1883.800905
2019-01-01 12:00:00+00:00  1917.337461
2019-01-01 13:00:00+00:00  1895.987749
2019-01-01 14:00:00+00:00  1892.995529
2019-01-01 15:00:00+00:00  1897.263752
2019-01-01 16:00:00+00:00  1920.733945
2019-01-01 17:00:00+00:00  1551.901840
2019-01-01 18:00:00+00:00  1306.198083
2019-01-01 19:00:00+00:00  2190.402477
2019-01-01 20:00:00+00:00  1600.062893
2019-01-01 21:00:00+00:00  1361.779141
2019-01-01 22:00:00+00:00  1424.458204
2019-01-01 23:00:00+00:00  1468.289676
2019-01-02 00:00:00+00:00  2135.030675
2019-01-02 01:00:00+00:00  1499.454806
2019-01-02 02:00:00+00:00  1827.869362
2019-01-02 03:00:00+00:00  1542.173913
2019-01-02 04:00:00+00:00  1871.024096
2019-01-02 05:00:00+00:00  1860.749625
2019-01

                                 power
datetime                              
2019-01-01 08:00:00+00:00  2394.690265
2019-01-01 09:00:00+00:00  2553.422222
2019-01-01 10:00:00+00:00  2358.839286
2019-01-01 11:00:00+00:00  2257.181208
2019-01-01 12:00:00+00:00  2378.495575
2019-01-01 13:00:00+00:00  2384.194260
2019-01-01 14:00:00+00:00  3114.696629
2019-01-01 15:00:00+00:00  3213.611691
2019-01-01 16:00:00+00:00  3229.227373
2019-01-01 17:00:00+00:00  2829.866667
2019-01-01 18:00:00+00:00  3029.155556
2019-01-01 19:00:00+00:00  2839.377778
2019-01-01 20:00:00+00:00  2866.216216
2019-01-01 21:00:00+00:00  2879.292035
2019-01-01 22:00:00+00:00  2917.847534
2019-01-01 23:00:00+00:00  2862.946429
2019-01-02 00:00:00+00:00  3169.464286
2019-01-02 01:00:00+00:00  2974.885655
2019-01-02 02:00:00+00:00  2960.000000
2019-01-02 03:00:00+00:00  2851.644444
2019-01-02 04:00:00+00:00  2762.389381
2019-01-02 05:00:00+00:00  2771.364653
2019-01-02 06:00:00+00:00  2667.280702
                         

                                  power
datetime                               
2019-01-01 08:00:00+00:00   6297.379310
2019-01-01 09:00:00+00:00   6376.000000
2019-01-01 10:00:00+00:00   6056.711111
2019-01-01 11:00:00+00:00   6743.294664
2019-01-01 12:00:00+00:00   9193.348115
2019-01-01 13:00:00+00:00  12948.277405
2019-01-01 14:00:00+00:00  12206.448598
2019-01-01 15:00:00+00:00  10948.666667
2019-01-01 16:00:00+00:00   8288.482143
2019-01-01 17:00:00+00:00   7287.770419
2019-01-01 18:00:00+00:00   5898.197802
2019-01-01 19:00:00+00:00   5361.415929
2019-01-01 20:00:00+00:00   5006.208426
2019-01-01 21:00:00+00:00   4999.727273
2019-01-01 22:00:00+00:00   5248.214286
2019-01-01 23:00:00+00:00   5903.482143
2019-01-02 00:00:00+00:00   7349.688889
2019-01-02 01:00:00+00:00   6812.024540
2019-01-02 02:00:00+00:00   7134.331066
2019-01-02 03:00:00+00:00   6187.363636
2019-01-02 04:00:00+00:00   6068.868778
2019-01-02 05:00:00+00:00   6402.379863
2019-01-02 06:00:00+00:00   6114.988610


In [ ]:
# GET HISTORICAL OR FORECAST PRICE FOR EACH BUILDING (UP TO 24 HOURS IN THE FUTURE)
# TODO: DOES NOT WORK address fails to bind
# GET THE TARIFF AND UTILITY FOR A GIVEN BUILDING
for bldg in buildings:
    print(bldg)
    tariff_util = xbos_services_getter.get_tariff_and_utility(price_stub,bldg)
    print(bldg,tariff_util)

# GET THE HISTORICAL PRICE FOR A GIVEN TARIFF AND UTILITY
# TODO return all available tariffs and utilities 
# TODO expose these calls in xbos_services_getter
# xbos_services_getter.get_price_utility_tariff()

# YOU CAN ALSO GET THE PRICE FOR A GIVEN BUILDING DIRECTLY
energy_price_type = 'ENERGY' 
demand_price_type = 'DEMAND'
for bldg in buildings:
    energy_price = xbos_services_getter.get_price(price_stub,bldg,energy_price_type,start,end,window)
    demand_price = xbos_services_getter.get_price(price_stub,bldg,demand_price_type,start,end,window)
    print(bldg)
    print(energy_price)
    print(demand_price)

In [ ]:
# GET HISTORICAL OUTDOOR TEMPERATURE FOR EACH BUILDING
for bldg in buildings:
    historical_outdoor_temperature = xbos_services_getter.get_outdoor_temperature_historic(outdoor_temperature_historical_stub,bldg,start,end,window)
    print(bldg)
    print(historical_outdoor_temperature)


In [ ]:
# GET HISTORICAL HVAC DATA FOR EACH ZONE IN EACH BUILDING 
# INDOOR TEMPERATURE, HVAC STATE, COOLING SETPOINT, HEATING SETPOING
for bldg in buildings:
    zones = xbos_services_getter.get_zones(building_zone_names_stub,bldg)
    for zone in zones:
        actions = xbos_services_getter.get_actions_historic(indoor_data_historical_stub,bldg,zone,start,end,window)
        indoor_temperatures = xbos_services_getter.get_indoor_temperature_historic(indoor_data_historical_stub,bldg,zone,start,end,window)
        #TODO add heating and cooling setpoints to microservice and to xbos-service-getter
        setpoints = xbos_services_getter.get_setpoints_historic(indoor_data_historical_stub,bldg,zone,start,end,window)
        print(bldg,zone)
#         print(actions)
#         print(indoor_temperatures)
        print(setpoints)
#         print(cooling_setpoints)

In [ ]:
# GET OCCUPANCY FOR EACH ZONE IN EACH BUILDING
for bldg in buildings:
    zones = xbos_services_getter.get_zones(building_zone_names_stub,bldg)
    for zone in zones:
        occupancy = xbos_services_getter.get_occupancy(occupancy_stub,bldg,zone,start,end,window)
        print(bldg,zone)
        print(occupancy)

In [ ]:
# GET COMFORT BAND AND DO NOT EXCEED BAND FOR EACH ZONE IN EACH BUILDING
for bldg in buildings:
    zones = xbos_services_getter.get_zones(building_zone_names_stub,bldg)
    for zone in zones:
        comfort_band = xbos_services_getter.get_comfortband(temperature_bands_stub,bldg,zone,start,end,window)
        do_not_exceed_band = xbos_services_getter.get_do_not_exceed(temperature_bands_stub,bldg,zone,start,end,window)
        print(bldg,zone)
        print(comfort_band)
        print(do_not_exceed_band)

In [ ]:
# GET A DISCOMFORT METRIC FOR A GIVEN TEMPERATURE AND A GIVEN COMFORT BAND
# TODO why do you require a building name as input??
temperature = 90.0 # Current temperature
temperature_low = 60 # 
temperature_high = 80
occupancy = 0.5 # 0.0 -> 1.0
discomfort = xbos_services_getter.get_discomfort(discomfort_stub,'bldg',temperature,temperature_low,temperature_high,occupancy)
print(discomfort)

In [ ]:
# GET HVAC CONSUMPTION FOR A GIVEN ZONE IN A GIVEN BUILDING
for bldg in buildings:
    zones = xbos_services_getter.get_zones(building_zone_names_stub,bldg)
    for zone in zones:
        hvac_consumption = xbos_services_getter.get_hvac_consumption(hvac_consumption_stub,bldg,zone)
        print(bldg,zone,hvac_consumption)

In [ ]:
# GET INDOOR TEMPERATURE PREDICTION FOR A ZONE IN A BUILDING GIVEN AN ACTION 
# (1-2 STAGE HEATING, 1-2 STAGAE COOLING, DO NOTHING)

action = 0 # 0, 1, 2, 3, 4
for bldg in buildings:
    outdoor_temperatures = xbos_services_getter.get_outdoor_temperature_historic(outdoor_temperature_historical_stub,bldg,start,end,window)
    outdoor_temperature = outdoor_temperatures[1]  
    zones = xbos_services_getter.get_zones(building_zone_names_stub,bldg)
    current_zones_temp = {}
    prev_zones_temp = {}
    for zone in zones:
        indoor_temp = xbos_services_getter.get_indoor_temperature_historic(indoor_data_historical_stub,bldg,zone,start,end,window)
        prev_zones_temp[zone] = indoor_temp[0]
        current_zones_temp[zone]= indoor_temp[1]
    for zone in zones:
        # THESE BUILDINGS DON'T HAVE A THERMAL MODEL
        if(bldg=='csu-dominguez-hills' or bldg=='local-butcher-shop' or bldg=='avenal-recreation-center'):
            continue
        print(bldg,zone)
#         temperature_prediction = xbos_services_getter.get_indoor_temperature_prediction(indoor_temperature_prediction_stub,bldg,zone,start,action,current_zones_temp[zone],outdoor_temperature,prev_zones_temp[zone],current_zones_temp)
        temperature_error_mean, temperature_error_var = xbos_services_getter.get_indoor_temperature_prediction_error(indoor_temperature_prediction_stub, bldg, zone, action, start, end)
#         print(temperature_prediction)
        print("Mean error", temperature_error_mean)
        print("Var error", temperature_error_var)
        
        

jesse-turner-center hvac_zone_resource_center
Mean error -0.0043703096053877314
Var error 0.011692033505848567
jesse-turner-center hvac_zone_class_132
Mean error -0.00046166278060831204
Var error 0.03501652152154823
jesse-turner-center hvac_zone_meeting_room_122
Mean error -0.003450541474633878
Var error 0.024360738889512405
jesse-turner-center hvac_zone_multi_purpose_147
Mean error -0.0031158655285149104
Var error 0.018291190602716098
jesse-turner-center hvac_zone_fitness_room_138
Mean error -0.00030817805199612777
Var error 0.012137068687874987
jesse-turner-center hvac_zone_assembly_112
Mean error -0.00021156044747910275
Var error 0.01534402193452032
jesse-turner-center hvac_zone_basketball_court_2
Mean error -6.921243036564652e-05
Var error 0.0038183700088557375
jesse-turner-center hvac_zone_basketball_court_6
Mean error -2.4460367093843217e-05
Var error 0.005031878536663339
jesse-turner-center hvac_zone_basketball_court_5
Mean error -1.761576135404536e-05
Var error 0.00374687900812

In [ ]:
# 
# TODO: USE OPTIMIZER with xbos-service getter